In [1]:
# Imporing libraries
import os
import pandas as pd
import numpy as np
import statistics as stat

## Read the dataset, display the first few rows to understand it, and display the count of ratings (rows) in the dataset to be sure that you download it correctly.

In [2]:
# Read the ratings dataset
baseLocation = "D:/TUNI/Courses/Period-2/DATA.ML.360 [Recommender]/Assignment 1/ml-latest-small/"
ratings_data = pd.read_csv(baseLocation+"ratings.csv")

# Displaying top 5 ratings data
ratings_data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
# Remove 'timestamp' column from ratings dataset because it's not needed.
ratings_data.drop('timestamp', inplace=True, axis=1)

# Displaying top 5 ratings data
ratings_data.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [4]:
# Count of ratings (rows)
print(len(ratings_data))

# Count of users
print(len(pd.unique(ratings_data['userId'])))

100836
610


In [5]:
# Read the movie dataset
movies_data = pd.read_csv(baseLocation+'movies.csv')

# Displaying top 5 movies data
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# Remove the "genres" column from the movies dataset because it is unnecessary.
movies_data.drop('genres', inplace=True, axis=1)

# Displaying top 5 ratings data
movies_data.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


## Implement the user-based collaborative filtering approach, using the Pearson correlation function for computing similarities between users, and the prediction function presented in class for predicting movies scores.

In [7]:
# Getting all the movies of a selected user(150) has seen and rated.
# This will be used as an input, and these values will be used to recommend some new movies for this user.
uId = 150
inputMovies = ratings_data[ratings_data['userId']== uId]
inputMovies.head(10)

,userId,movieId,rating
22277,150,3,3.0
22278,150,5,3.0
22279,150,6,4.0
22280,150,7,3.0
22281,150,25,4.0
22282,150,32,5.0
22283,150,36,4.0
22284,150,52,4.0
22285,150,58,3.0
22286,150,62,3.0


In [8]:
# Number of movies that user(150) has rated or watched
len(inputMovies)

26

In [9]:
# Identifying users who rate films similarly to our chosen user(150)
similarUsers = ratings_data[ratings_data['movieId'].isin(inputMovies['movieId'].tolist())]
similarUsers

,userId,movieId,rating
1,1,3,4.0
2,1,6,4.0
36,1,608,5.0
37,1,648,3.0
40,1,733,4.0
...,...,...,...
99537,610,32,4.5
99541,610,95,3.5
99576,610,608,4.5
99582,610,780,3.5


In [10]:
# Creating sub dataframes based on userId
similarUsersGroup = similarUsers.groupby(['userId'])

In [11]:
# Displaying random user, suppose userId = 580
similarUsersGroup.get_group(580)

,userId,movieId,rating
89317,580,6,4.0
89321,580,25,4.0
89322,580,32,5.0
89326,580,62,2.0
89366,580,608,4.0
89367,580,653,3.0
89374,580,780,3.0
89427,580,1356,4.0


In [12]:
# Sort users dataframes
similarUsersGroup = sorted(similarUsersGroup, key = lambda x:len(x[1]), reverse = True)

In [13]:
# Displaying the 2nd top user (1st one is the same as our selected user)
similarUsersGroup[1]

(270,
        userId  movieId  rating
 39061     270        3     3.0
 39062     270        5     3.0
 39063     270        6     3.0
 39064     270        7     3.0
 39068     270       25     4.0
 39069     270       32     4.0
 39070     270       36     5.0
 39071     270       52     4.0
 39072     270       58     4.0
 39073     270       62     5.0
 39074     270       79     2.0
 39075     270       95     2.0
 39080     270      141     3.0
 39081     270      376     4.0
 39082     270      494     2.0
 39083     270      608     4.0
 39084     270      628     3.0
 39086     270      648     3.0
 39087     270      653     2.0
 39090     270      733     3.0
 39094     270      780     4.0
 39095     270      784     1.0
 39096     270      786     3.0
 39098     270     1073     5.0
 39099     270     1356     4.0)

In [14]:
# Number of similar users, who rated all the movies or some of them.
len(similarUsersGroup)

414

In [15]:
# userId of the 2nd top user
similarUsersGroup[1][0]

270

In [16]:
# Dataframe of the 2nd top user
similarUsersGroup[1][1]

,userId,movieId,rating
39061,270,3,3.0
39062,270,5,3.0
39063,270,6,3.0
39064,270,7,3.0
39068,270,25,4.0
39069,270,32,4.0
39070,270,36,5.0
39071,270,52,4.0
39072,270,58,4.0
39073,270,62,5.0


In [17]:
#let's check how many movies this user has rated
len(similarUsersGroup[1][1])

25

### Calculating The Similarity (Pearson Correlation) of the Selected (Input) User to the Similar User

In [18]:
similarUsersGroup = similarUsersGroup[1:101] # Removing the first user (userId =150) because it is our selected user
len(similarUsersGroup)

100

In [19]:
# Store the Pearson Correlation in a dictionary
pearsonCorrelationDict = {}

# For every similar user group in our subset
for name, group in similarUsersGroup:
    
    # Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    
    # Find out the Selected User's rating for the movies they both like
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    
    # Store them in a temporary buffer variable
    selectedUserTempRatingList = temp_df['rating'].tolist()
    
    # Create a list of the most recent user group reviews
    similarUserTempRatingList = group['rating'].tolist()
    
    # Calculating pearson similarity
    simXX = 0
    meanSelectedUserTempRating = stat.mean(selectedUserTempRatingList)
    for i in selectedUserTempRatingList:
        simXX = simXX + pow((i - meanSelectedUserTempRating),2)
    
    simYY = 0
    meanSimilarUserTempRaning = stat.mean(similarUserTempRatingList)
    for j in similarUserTempRatingList:
        simYY = simYY + pow((j - meanSimilarUserTempRaning),2)
        
    simXY = 0
    for i, j in zip(selectedUserTempRatingList, similarUserTempRatingList):
        simXY = simXY+ ((i - meanSelectedUserTempRating ) * (j-meanSimilarUserTempRaning))
    
    # If the denominator is different than zero, then divide, else, 0 correlation.
    if simXX != 0 and simYY != 0:
        pearsonCorrelationDict[name] = simXY/np.sqrt(simXX*simYY)
    else:
        pearsonCorrelationDict[name] = 0                    

In [20]:
pearsonCorrelationDict.items()

dict_items([(270, 0.3138439323974278), (337, -0.18158301940785762), (414, 0.23757330776262545), (492, 0.07811050188185686), (32, 0.24342022732490098), (599, 0.6081743121036026), (91, 0.5048561696263706), (389, 0.37401122050492525), (437, 0.37824025887080936), (474, 0.7010455033329385), (521, 0.2869999276570522), (68, 0.33752495789470566), (451, 0.40551769129747745), (6, 0.12788168365775585), (117, 0.014782809899727049), (206, -0.04423739552038095), (470, -0.05455447255899805), (84, 0.35010198941225656), (269, 0.4885042104591971), (304, 0.049225410069563956), (314, -0.22490326100567096), (350, 0.40327956630872147), (501, 0.09439751632913282), (559, 0.17309692896316617), (590, 0.6569344101227067), (602, -0.0704504917567163), (57, 0.5708992257184501), (179, 0.4237017130659009), (217, 0.22310332740004063), (274, 0.6214145809051803), (364, -0.6770032003863303), (448, 0.053300179088902624), (489, 0.4748843559385291), (600, 0.3056090519853218), (64, 0.0), (90, -0.3104602102825331), (160, 0.39

In [21]:
similarityScoreofSimilarUsers_data = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
similarityScoreofSimilarUsers_data.head()

,0
270,0.313844
337,-0.181583
414,0.237573
492,0.078111
32,0.243420


In [22]:
# Similarity score of other similar users to our selected user (150)
similarityScoreofSimilarUsers_data.columns = ['similarityScore']
similarityScoreofSimilarUsers_data['userId'] = similarityScoreofSimilarUsers_data.index
similarityScoreofSimilarUsers_data.index = range(len(similarityScoreofSimilarUsers_data))
similarityScoreofSimilarUsers_data.head()

,similarityScore,userId
0,0.313844,270
1,-0.181583,337
2,0.237573,414
3,0.078111,492
4,0.243420,32


## The top 10 similar users

In [23]:
topSimilarUsers=similarityScoreofSimilarUsers_data.sort_values(by='similarityScore', ascending=False)
topSimilarUsers.head(10)

,similarityScore,userId
98,0.872082,11
85,0.842701,239
96,0.825137,580
87,0.821584,297
74,0.793857,191
86,0.790906,277
55,0.772099,380
94,0.721688,477
57,0.701502,18
9,0.701046,474


### Calculating Recommendation Score

In [24]:
topSimilarUsersRating = topSimilarUsers.merge(ratings_data, left_on='userId', right_on='userId', how='inner')
topSimilarUsersRating.head()

,similarityScore,userId,movieId,rating
0,0.872082,11,6,5.0
1,0.872082,11,10,3.0
2,0.872082,11,36,4.0
3,0.872082,11,44,2.0
4,0.872082,11,95,3.0


In [25]:
# Calculating the mean ratings
meanRb = topSimilarUsersRating.groupby('userId').mean()[['rating']]
meanRb.columns = ['avgRating']
meanRb['userId'] = meanRb.index
meanRb.index = range(len(meanRb))
meanRb

,avgRating,userId
0,4.366379,1
1,3.493631,6
2,3.781250,11
3,3.732072,18
4,2.607397,19
...,...,...
95,2.642050,599
96,2.991481,600
97,3.392593,602
98,3.507953,603


In [26]:
topSimilarUsersRating = topSimilarUsersRating.merge(meanRb, left_on='userId', right_on='userId', how='inner')
topSimilarUsersRating.head()

,similarityScore,userId,movieId,rating,avgRating
0,0.872082,11,6,5.0,3.78125
1,0.872082,11,10,3.0,3.78125
2,0.872082,11,36,4.0,3.78125
3,0.872082,11,44,2.0,3.78125
4,0.872082,11,95,3.0,3.78125


In [27]:
topSimilarUsersRating['weightedRatingScore'] = topSimilarUsersRating['similarityScore']*(topSimilarUsersRating['rating']-topSimilarUsersRating['avgRating'])
topSimilarUsersRating.head()

,similarityScore,userId,movieId,rating,avgRating,weightedRatingScore
0,0.872082,11,6,5.0,3.78125,1.062849
1,0.872082,11,10,3.0,3.78125,-0.681314
2,0.872082,11,36,4.0,3.78125,0.190768
3,0.872082,11,44,2.0,3.78125,-1.553395
4,0.872082,11,95,3.0,3.78125,-0.681314


In [28]:
tempTopSimilarUsersRating = topSimilarUsersRating.groupby('movieId').sum()[['weightedRatingScore']]
tempTopSimilarUsersRating.columns = ['sum_weightedRatingScore']
tempTopSimilarUsersRating['movieId'] = tempTopSimilarUsersRating.index
tempTopSimilarUsersRating.head()

,sum_weightedRatingScore,movieId
movieId,,
1,7.770884,1
2,-2.651452,2
3,-2.703619,3
4,-0.760304,4
5,-6.762971,5


In [29]:
# Creates an empty dataframe
recommendation_data = pd.DataFrame()

# Weighted average
meanRa = inputMovies['rating'].mean()
recommendation_data['weighted average recommendation score'] = meanRa+(tempTopSimilarUsersRating['sum_weightedRatingScore']/topSimilarUsers['similarityScore'].sum())
recommendation_data['movieId'] = recommendation_data.index
recommendation_data.head()

,weighted average recommendation score,movieId
movieId,,
1,3.922200,1
2,3.459113,2
3,3.456795,3
4,3.543141,4
5,3.276430,5


### Recommended Movies 

In [30]:
recommendation_data = recommendation_data.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_data.head(20)

,weighted average recommendation score,movieId
movieId,,
296,4.473528,296
260,4.354835,260
32,4.345132,32
2571,4.302222,2571
1196,4.290965,1196
318,4.288756,318
1210,4.271334,1210
2028,4.251648,2028
50,4.246021,50


## Top 20 Recommended Movies

In [31]:
# Top 20 Recommended Movies
movies_data.loc[movies_data['movieId'].isin(recommendation_data.head(20)['movieId'].tolist())]

,movieId,title
5,6,Heat (1995)
31,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
46,50,"Usual Suspects, The (1995)"
224,260,Star Wars: Episode IV - A New Hope (1977)
254,293,Léon: The Professional (a.k.a. The Professiona...
257,296,Pulp Fiction (1994)
277,318,"Shawshank Redemption, The (1994)"
314,356,Forrest Gump (1994)
507,589,Terminator 2: Judgment Day (1991)
510,593,"Silence of the Lambs, The (1991)"
